In [100]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [101]:
file_path='../input/latest-laptop-price-list/Laptop.csv'
file_path_cleaned= '../input/latest-laptop-price-list/Cleaned_Laptop_data.csv'
laptop = pd.read_csv(file_path)
laptop_cleaned=pd.read_csv(file_path_cleaned)

In [102]:
laptop.head()

In [103]:

del laptop['Unnamed: 0']

laptop.columns


In [104]:
laptop.shape[0]


In [105]:
laptop.info()

In [106]:
laptop.Brand.unique()

In [107]:
laptop['Model']=laptop['Model'].str.lower()

In [108]:
laptop['Model'].unique()

In [109]:
laptop['Processor'].unique()
#laptop['Processor_Gen'].unique()

In [110]:
#creating a new column indication processor brand
laptop['Processor1']=laptop['Processor']

def processor_alter(p):
    if p == 'i3' or p== 'i5' or p=='i7': 
        p = 'Intel'
        
    elif p == '3250U' or p== '3050U' or p== '5600U' or p=='5500U' or p== '5300U' or p == '5600H' or p == '3450U' or p=='4800HS' or p== '4600H' or p== '4800H' or p=='R3-3250U':
        p = 'AMD'
        
    else: 
        p= 'Others'
        
    return p

laptop['Processor1'] = laptop['Processor1'].apply(processor_alter)
        
laptop = laptop.rename(columns = {'Processor1':'Processor_brand'})
#laptop.Processor = laptop['Model'].apply(lambda x: 'AMD' if x =='vivobook' or x =='ryzen' else 'Celeron')



In [111]:

#replace incorrect data 

laptop[(laptop.Processor == '-') & (laptop.Model == 'vivobook')] = laptop[(laptop.Processor == '-') & (laptop.Model == 'vivobook')].replace('-','Others')

laptop[(laptop.Processor == '-')]=laptop[(laptop.Processor == '-')].replace('-','Others')



In [112]:
laptop.groupby('OS').count()

In [113]:
laptop[laptop.OS.isin(['128 GB SSD','256 GB SSD'])]



In [114]:
#because the values in 'OS' with a hard disk format could be indicating that this device has a second hard disk installed
#we will create a column to represent that

laptop['2nd_hard_disk'] = laptop['OS']

laptop['2nd_hard_disk']=laptop['2nd_hard_disk'].replace(['Chrome OS','DOS','Windows 10','Windows 10 Home','Windows 11 Home'], np.NaN)

laptop.head()

In [115]:
#replace the incorrect value of 'acer swift 3' with correct values, because it was the only
#record with '128 GB SSD' for 'OS' (record 106)

laptop.OS= laptop.OS.replace(to_replace ='128 GB SSD', value = 'Windows 10')



In [116]:
#the OS for these devices could be either windows 10, windows 10 Home or windows 11 Home, except for one specific device 
#with 4GB RAM which cannot handle windows 11 Home. Hence, we will fill in random values of the three except for that device 

os_list = ['Windows 10 Home', 'Windows 11 Home', 'Windows 10']

def os_alter(OS):
    if OS == '256 GB SSD':
        OS = np.random.choice(os_list)
    return OS

laptop.OS= laptop['OS'].apply(os_alter)


In [117]:
#Correct Price data value
laptop.Price = laptop['Price']*0.01

In [118]:
#looking for any NA values in the 'OS' column

laptop[laptop.OS.isna()]

In [119]:
#the apple device will have 'MacOS' as its 'OS'
laptop.OS = laptop.OS.replace(np.nan, 'MacOS')
laptop[laptop.OS == 'MacOS']


#there is one DELL device that need adjustment, the index of it is 70
laptop[laptop.index == 70] = laptop[laptop.index == 70].replace('MacOS','Windows 10 Home')



In [147]:
#Exploratory Analysis

In [121]:
#create a sub dataframe for visualization - model count of each brand
brand_count=laptop.groupby('Brand')[['Model']].count()
brand_count=brand_count.sort_values('Model',ascending = False)

#import matplotlib to creat visualization
import matplotlib.pyplot as plt
%matplotlib inline

#visualization creating
viz1=brand_count.plot(kind='bar',figsize= (12,8))

#set graph title
plt.title('Model count of each Brand')

#set data lable for the graph 
viz1.bar_label(viz1.containers[0],label_type='edge');

#the top 3 most count brands were DELL, ASUS and Lenovo


In [122]:
#create sub dataframe for visualization
processor_c = laptop.groupby('Processor')[['Model']].count()
processor_c = processor_c.sort_values('Model',ascending = False)

#create a bar graph
viz2=processor_c.plot(kind='bar', figsize=(12,8))

#setting graph title
plt.title('Processor popularity')

#creating data label for the graph
viz2.bar_label(viz2.containers[0], label_type='edge', color='black',rotation= 90, fontsize=8, padding = 3);


#the i3 and i5 cores took dominance among the laptops in the dataset


In [123]:
#pie chart for Ram size 
#sub dataframe for the pie chart 
ram_size = laptop.groupby('RAM')[['Brand']].count()

#pie chart creating 
ram_pie = ram_size.plot(kind='pie',y='Brand', figsize=(5,5),autopct='%.1f%%')

#set up chart's name
ram_pie=plt.title('RAM size (GB) popularity over total device count')


In [142]:
#My brother wants me to find him a laptop with good performance for his study and gaming with a budget below 1000$ from this data set. Now to find the attributes which would lead to a worthy choice of laptop
#we will have to figure out which is the most popular price appeared in the dataset to have a bigger possibility to find a good laptop
#if the said price is lower than the mean price (800$) we will accept it because or budget is below 1000$

import seaborn as sns
sns.distplot(laptop['Price'])
plt.show()

#Laptop price is skewed to the left side of the distribution. Laptop are sold most at the price of 400$, which is an affordable price

In [143]:
#Price vs Name brand
sns.barplot(x=laptop['Brand'],y=laptop['Price'])
plt.xticks(rotation='vertical')
plt.show()

#apple & microsoft has a very high average price, only apple goes above 1000$ price point
#so we will continue with the rest of the brand for our selection except for apple

In [144]:
#does price varies in term of its rating 
sns.scatterplot(x=laptop['Rating'],y=laptop['Price'])

#the most dense rating group is around 4.00 to 4.625 of Rating and 300$ to 800$ in Price. This proves that
#price does not affect the rating . Instead, it will be more informative if we study this sample size of Price range
#to find the best latop 

In [145]:
#does Ram size determine the Price ?
sns.scatterplot(x=laptop['RAM'], y=laptop['Price']);

#Yes, the higher the RAM size the more expensive. Since we also put performance into consideration
#we will look into devices with 8GB or above


In [141]:
#does Hard disk size has impact on the Price ?
sns.scatterplot(x=laptop['Hard_Disk_Capacity'], y=laptop['Price']);
sns.set(rc={'figure.figsize':(11.7,16)})

#No, the hard disk capacity does not have an critical impact on the price of the laptop.Most of the population lies on 1TB HDD, 512GB SSD, 256GB SSD. Since having an SSD would be better nowadays
#we will select 512 GB SSD as our favorable attribute since we value performance of the device

In [146]:
#does Processor brand determine Price ? 
sns.barplot(x=laptop['Processor_brand'],y=laptop['Price'])
plt.xticks(rotation='vertical')
plt.show()

#AMD and Intel has the same price range while Intel has a more narrow distribution over the price range. So device with 
#Intel chip will usually come with an affordable price

In [ ]:
#in sum, we have narrow down our selected attribute to find the best laptop for my brother
#the select population of laptop for further review is Intel chip, 512 GB SSD, is from any 
#brand except for Apple, has RAM size above 8GB and has rating above 4.0

In [150]:
ideal_laptop=laptop[(laptop.Processor_brand == 'Intel')&
                    (laptop.Price < 1000)&
                    (laptop.Hard_Disk_Capacity == '512 GB SSD')&
                    (laptop.Brand !='Apple')&
                    (laptop.RAM > 8.0)
                      & (laptop.Rating > 4.0)]
ideal_laptop

In [160]:
#we will calculate the cost efficient ratio among these ideal laptop to find the perfect laptop
#for my brother by calculating Price/Rating

ideal_laptop['CE_ratio']=ideal_laptop['Price']/ideal_laptop['Rating']

#the laptop to buy is:
ideal_laptop.sort_values('CE_ratio',ascending = False).head(1)